# Notebook for Formatting Data Tables into Busses and Edges

### Preparation

In [1]:
import pandas as pd
import numpy as np

# Read Files
demanders = pd.read_excel('Busses_Timeseries_Table.xlsx')
generators = pd.read_excel('Generators_Timeseries_Table.xlsx')
generatorsGrid = pd.read_excel('Generators_Table.xlsx')
branches = pd.read_excel('Branches_Input_Table.xlsx')

In [4]:
# Rename and choose relevant Columns in Generators Table
generatorsGrid['generator_id'] = generatorsGrid.index
generatorsGrid.rename(columns = {'Number of Bus':'bus_id'}, inplace = True)
generators['generator_id'] = pd.factorize(generators['unit_code'])[0]
generators = pd.merge(generators, generatorsGrid[['generator_id','bus_id']], how='left', on='generator_id').fillna(0)
generators.rename(columns = {'bid_datetime_beginning_utc':'datetime_beginning_utc'}, inplace = True)
generators

,datetime_beginning_utc,bid_datetime_beginning_ept,unit_code,bid_slope_flag,mw1,mw2,mw3,mw4,mw5,mw6,...,min_ecomax,avg_ecomax,max_ecomin,min_ecomin,avg_ecomin,incremental_price,solGenerate,solOn,generator_id,bus_id
0,2021-06-01 04:00:00,2021-06-01 00:00:00,AAAABg8LDwQAMS45NjYyMjEwNA,True,1.0,749,1168.0,1169.0,1484.0,0.0,...,810.0,810.0,810.0,81.0,810.0,21.505539,202.50,1.0,0,0
1,2021-06-01 04:00:00,2021-06-01 00:00:00,AAAADQYJAgABLjEwNTAyMjE0,True,0.0,25,48.9,51.3,0.0,0.0,...,46.9,46.9,25.0,25.0,25.0,44.652378,46.90,1.0,1,1
2,2021-06-01 04:00:00,2021-06-01 00:00:00,AAAADQYJAgADLjEwNTAyMjEz,True,0.0,25,48.9,51.3,0.0,0.0,...,46.9,46.9,25.0,25.0,25.0,44.652378,46.90,1.0,2,4
3,2021-06-01 04:00:00,2021-06-01 00:00:00,AAAADQYJAgAFLjEwNTAyMjEy,True,0.0,25,48.9,51.3,0.0,0.0,...,46.9,46.9,25.0,25.0,25.0,44.652378,45.61,1.0,3,7
4,2021-06-01 04:00:00,2021-06-01 00:00:00,AAAADQYJAgAHLjEwNTAyMjEw,True,0.0,25,48.6,50.7,0.0,0.0,...,46.9,46.9,25.0,25.0,25.0,46.112308,0.00,0.0,4,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52699,2022-06-02 03:00:00,2022-06-01 23:00:00,AAAADQYJAgABLjEwNTAyMjE0,True,0.0,25,48.9,51.1,0.0,0.0,...,46.9,46.9,25.0,25.0,25.0,92.955421,46.90,1.0,1,1
52700,2022-06-02 03:00:00,2022-06-01 23:00:00,AAAADQYJAgADLjEwNTAyMjEz,True,0.0,25,48.9,51.1,0.0,0.0,...,46.9,46.9,25.0,25.0,25.0,92.955421,46.90,1.0,2,4
52701,2022-06-02 03:00:00,2022-06-01 23:00:00,AAAADQYJAgAFLjEwNTAyMjEy,True,0.0,25,48.9,51.1,0.0,0.0,...,46.9,46.9,25.0,25.0,25.0,92.955421,46.90,1.0,3,7
52702,2022-06-02 03:00:00,2022-06-01 23:00:00,AAAADQYJAgAHLjEwNTAyMjEw,True,0.0,25,49.0,51.1,0.0,0.0,...,46.9,46.9,25.0,25.0,25.0,103.009667,35.06,1.0,4,10


### Merge

In [239]:
#Merge generators and busses
busses = pd.merge(demanders[['datetime_beginning_utc', 'bus_id', 'load_area', 'mw_norm', 'solTheta']], generators[['datetime_beginning_utc', 'bus_id', 'inter_start_cost', 'incremental_price', 'min_ecomin', 'max_ecomax', 'solGenerate', 'solOn']], how='left', on=['datetime_beginning_utc','bus_id']).fillna(0)


In [ ]:
#Merge edges table
timestamps = generators['datetime_beginning_utc'].unique()
# Cross Join to turn grid into timeseries data (same grid)
timeDF = pd.DataFrame(timestamps, columns=['datetime_beginning_utc'])
edges = pd.merge(timeDF[['datetime_beginning_utc']], branches[['From Number', 'To Number', 'X', 'Lim MVA A']], how='cross')

In [17]:
busses

,datetime_beginning_utc,bus_id,load_area,mw_norm,solTheta,inter_start_cost,incremental_price,min_ecomin,max_ecomax,solGenerate,solOn
0,2021-06-01 04:00:00,0,AECO,1.17,4292.559368,20427.25,21.505539,40.5,101.25,202.5,1.0
1,2021-06-01 04:00:00,1,AEPAPT,8.83,2151.492013,52.50,44.652378,25.0,46.90,46.9,1.0
2,2021-06-01 04:00:00,2,AEPIMP,6.65,3826.163320,0.00,0.000000,0.0,0.00,0.0,0.0
3,2021-06-01 04:00:00,3,AEPKPT,1.38,1968.679755,0.00,0.000000,0.0,0.00,0.0,0.0
4,2021-06-01 04:00:00,4,AEPOPT,13.82,2130.360360,52.50,44.652378,25.0,46.90,46.9,1.0
...,...,...,...,...,...,...,...,...,...,...,...
263515,2022-06-02 03:00:00,25,RECO,0.32,680.416156,0.00,0.000000,0.0,0.00,0.0,0.0
263516,2022-06-02 03:00:00,26,SMECO,1.03,256.304077,0.00,0.000000,0.0,0.00,0.0,0.0
263517,2022-06-02 03:00:00,27,UGI,0.26,565.551708,0.00,0.000000,0.0,0.00,0.0,0.0
263518,2022-06-02 03:00:00,28,VMEU,0.20,122.315316,0.00,0.000000,0.0,0.00,0.0,0.0


### Storage

In [32]:
busses.to_csv("Busses_Timeseries_Table.csv")
edges.to_csv("Edges_Timeseries_Table.csv")